In [17]:
# СОСНЮК КАТЕРИНА ФІТ 4-6 ВАРІАНТ 22 
# Створити RNN для семантичного аналізу тексту з використанням моделі LSTM
import numpy as np
import re
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

texts_true = [
    "Це текст, який містить позитивну інформацію.",
    "Ще один приклад позитивного тексту.",
    "Тексти можуть містити різні емоції, включаючи позитивні."
]

texts_false = [
    "Це текст, який містить негативну інформацію.",
    "Не всі тексти будуть позитивними.",
    "Можна зустріти і негативні тексти."
]


texts = texts_true + texts_false
labels = [1] * len(texts_true) + [0] * len(texts_false)  

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, lower=True)
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(sequences, maxlen=10)


Y = np.array(labels)
Y = np.vstack([1 - Y, Y]).T  


indices = np.arange(data_pad.shape[0])
np.random.shuffle(indices)
data_pad = data_pad[indices]
Y = Y[indices]


model = Sequential([
    Embedding(maxWordsCount, 64),
    Dropout(0.5),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)


history = model.fit(data_pad, Y, batch_size=32, epochs=50, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


def predict_sentiment(text):
    text_seq = tokenizer.texts_to_sequences([text.lower()])
    text_pad = pad_sequences(text_seq, maxlen=10)
    prediction = model.predict(text_pad)
    sentiment = np.argmax(prediction)
    return 'Positive' if sentiment == 1 else 'Negative'



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.6889 - val_accuracy: 0.5000 - val_loss: 0.6934 - learning_rate: 0.0010
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.6883 - val_accuracy: 0.5000 - val_loss: 0.6935 - learning_rate: 0.0010
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7500 - loss: 0.6892 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 0.0010
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7500 - loss: 0.6869 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 2.0000e-04
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7500 - loss: 0.6920 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 2.0000e-04
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 0.6819 - val_accuracy: 0.0000e+00 - val_loss: 0.6936 - learning_rate: 1.0000e-04


In [21]:
res = predict_sentiment("Можна зустріти і негативні тексти.")
if res == 'Positive':
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Результат. Текст негативний.


In [20]:
res = predict_sentiment("Тексти можуть містити різні емоції, включаючи позитивні.")
if res == 'Positive':
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Результат. Текст позитивний.


In [ ]:
#ВИСНОВКИ: БУЛО СТВОРЕНО RNN ДЛЯ СЕМАНТИЧНОГО АНАЛІЗУ ТЕКСТУ З ВИКОРИСТАННЯМ МОДЕЛІ LSTM, ОТРИМАНО ПОЗИТИВНИЙ ТА НЕГАТИВНИЙ ВІДГУКИ